The system trains BERT (or any other transformer model like RoBERTa, DistilBERT etc.) on the SNLI + MultiNLI (AllNLI) dataset
with MultipleNegativesRankingLoss. Entailnments are poisitive pairs and the contradiction on AllNLI dataset is added as a hard negative.
At every 10% training steps, the model is evaluated on the STS benchmark dataset

In [1]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 105.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 127.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=1efb05a76953d6f002efb3c37f0bb5b47c56af7890ca2b76999554871d9a6f11
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
from sentence_transformers import SentenceTransformer, util
from sentence_transformers import models, losses, datasets
from sentence_transformers import LoggingHandler, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from torch.utils.data import DataLoader
import math
import random
import logging
from datetime import datetime
import os
import gzip
import csv

In [3]:
#Check if dataset exsist. If not, download and extract  it
nli_dataset_path = 'data/AllNLI.tsv.gz'
sts_dataset_path = 'data/stsbenchmark.tsv.gz'

if not os.path.exists(nli_dataset_path):
  util.http_get('https://sbert.net/datasets/AllNLI.tsv.gz', nli_dataset_path)

if not os.path.exists(sts_dataset_path):
  util.http_get('https://sbert.net/datasets/stsbenchmark.tsv.gz', sts_dataset_path)

def add_to_samples(sent1,sent2,label):
  if sent1 not in train_data:
    train_data[sent1] = {'contradiction':set(),'entailment':set(),'neutral':set()}
  train_data[sent1][label].add(sent2)

train_data = {}
with gzip.open(nli_dataset_path,'rt',encoding='utf8') as f:
  reader = csv.DictReader(f,delimiter ='\t',quoting=csv.QUOTE_NONE)
  for row in reader:
    if row['split'] =='train':
      sent1 = row['sentence1'].strip()
      sent2 = row['sentence2'].strip()

      add_to_samples(sent1,sent2,row['label'])
      add_to_samples(sent2,sent1,row['label']) # add the opposite

train_data

  0%|          | 0.00/40.8M [00:00<?, ?B/s]

  0%|          | 0.00/392k [00:00<?, ?B/s]

{'A person on a horse jumps over a broken down airplane.': {'contradiction': {'A person is at a diner, ordering an omelette.'},
  'entailment': {'A person is outdoors, on a horse.'},
  'neutral': {'A person is training his horse for a competition.'}},
 'A person is training his horse for a competition.': {'contradiction': set(),
  'entailment': set(),
  'neutral': {'A person on a horse jumps over a broken down airplane.'}},
 'A person is at a diner, ordering an omelette.': {'contradiction': {'A person on a horse jumps over a broken down airplane.'},
  'entailment': set(),
  'neutral': set()},
 'A person is outdoors, on a horse.': {'contradiction': set(),
  'entailment': {'A person on a horse jumps over a broken down airplane.'},
  'neutral': set()},
 'Children smiling and waving at camera': {'contradiction': {'The kids are frowning'},
  'entailment': {'There are children present'},
  'neutral': {'They are smiling at their parents'}},
 'They are smiling at their parents': {'contradictio

In [5]:
model_name = "distilroberta-base"
train_batch_size = 128          #The larger you select this, the better the results (usually). But it requires more GPU memory
max_seq_length = 75
num_epochs = 3

# Save path of model
model_path = 'output/training_nli'

# Here we define our SentenceTransformer model
word_embedding_model = models.Transformer(model_name,max_seq_length=max_seq_length)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),pooling_mode='mean')
model = SentenceTransformer(modules=[word_embedding_model,pooling_model])

train_samples = []
for sent1,others in train_data.items():
  if len(others['entailment']) > 0 and len(others['contradiction']) > 0:
    train_samples.append(InputExample(texts =[sent1,random.choice(list(others['entailment'])),random.choice(list(others['contradiction']))]))
    train_samples.append(InputExample(texts=[random.choice(list(others['entailment'])),sent1,random.choice(list(others['contradiction']))]))

# Special data loader that avoid duplicates within a batch
train_dataloader = datasets.NoDuplicatesDataLoader(train_samples[:40000],batch_size=train_batch_size)

# training loss
train_loss = losses.MultipleNegativesRankingLoss(model)

dev_samples = []
with gzip.open(sts_dataset_path,'rt',encoding='utf8') as f:
  reader = csv.DictReader(f,delimiter='\t',quoting=csv.QUOTE_NONE)
  for row in reader:
    if row['split'] == 'dev':
      score = float(row['score'])/5.0
      dev_samples.append(InputExample(texts=[row['sentence1'],row['sentence2']],label=score))

dev_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples,batch_size=train_batch_size,name='sts-dev')

# Configure the training
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)

# Train model
model.fit(train_objectives = [(train_dataloader,train_loss)],
          evaluator = dev_evaluator,
          epochs=num_epochs,
          evaluation_steps = int(len(train_dataloader)*0.1),
          warmup_steps = warmup_steps,
          output_path = model_path,
          use_amp=False #Set to True, if your GPU supports FP16 operations
          )

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/312 [00:00<?, ?it/s]

Iteration:   0%|          | 0/312 [00:00<?, ?it/s]

Iteration:   0%|          | 0/312 [00:00<?, ?it/s]

In [6]:
# Load the stored model and evaluate its performance on STS benchmark dataset

test_samples = []
with gzip.open(sts_dataset_path, 'rt', encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        if row['split'] == 'test':
            score = float(row['score']) / 5.0 #Normalize score to range 0 ... 1
            test_samples.append(InputExample(texts=[row['sentence1'], row['sentence2']], label=score))

model = SentenceTransformer(model_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, batch_size=train_batch_size, name='sts-test')
test_evaluator(model, output_path=model_path)

0.7864150242667373